In [1]:
import numpy as np
from numba import jit, prange
from time import *

In [18]:
def Kadene_extended(Arr):
    ind0_starting_here = 0
    ind0_max_so_far = 0
    L = 0
    max_so_far = Arr[0]
    max_ending_here = 0
    
    for i in range(len(Arr)):
        max_ending_here += Arr[i]
        if max_ending_here < 0:
            max_ending_here = 0
            ind0_starting_here = i + 1
            
        elif max_so_far < max_ending_here:
            max_so_far = max_ending_here
            ind0_max_so_far = ind0_starting_here
            L = i + 1 - ind0_max_so_far
            
        elif max_so_far == max_ending_here:
            if i + 1 - ind0_starting_here > L:
                ind0_max_so_far = ind0_starting_here
                L = i + 1 - ind0_max_so_far
                
    if max_so_far > 0:
        return max_so_far, Arr[ind0_max_so_far:ind0_max_so_far + L]
    else:
        return 0, []

In [22]:
@jit(nopython = True)
def Kadane_numba(Arr):

    ind0_starting_here = 0
    ind0_max_so_far = 0
    L = 0
    max_so_far = Arr[0]
    max_ending_here = 0
    
    for i in range(len(Arr)):
        max_ending_here += Arr[i]
        if max_ending_here < 0:
            max_ending_here = 0
            ind0_starting_here = i + 1
            
        elif max_so_far < max_ending_here:
            max_so_far = max_ending_here
            ind0_max_so_far = ind0_starting_here
            L = i + 1 - ind0_max_so_far
            
        elif max_so_far == max_ending_here:
            if i + 1 - ind0_starting_here > L:
                ind0_max_so_far = ind0_starting_here
                L = i + 1 - ind0_max_so_far
                
    if max_so_far > 0:
        return max_so_far, Arr[ind0_max_so_far:ind0_max_so_far + L]

In [38]:
@jit(nopython = True)
def Kadane_job(Arr, N):
    if N >= len(Arr):
        N = len(Arr)
        
    ind0_starting_here = 0
    ind0_max_so_far = 0
    L = 0
    max_so_far = Arr[0]
    max_ending_here = 0
    
    for i in range(N):
        max_ending_here += Arr[i]
        if max_ending_here < 0:
            max_ending_here = 0
            ind0_starting_here = i + 1
            
        elif max_so_far < max_ending_here:
            max_so_far = max_ending_here
            ind0_max_so_far = ind0_starting_here
            L = i + 1 - ind0_max_so_far
            
        elif max_so_far == max_ending_here:
            if i + 1 - ind0_starting_here > L:
                ind0_max_so_far = ind0_starting_here
                L = i + 1 - ind0_max_so_far
                
    i = N
    if i < len(Arr):
        max_ending_here += Arr[i]
        
        while max_ending_here >= 0:

            if max_ending_here < 0:
                max_ending_here = 0
                ind0_starting_here = i + 1

            elif max_so_far < max_ending_here:
                max_so_far = max_ending_here
                ind0_max_so_far = ind0_starting_here
                L = i + 1 - ind0_max_so_far

            elif max_so_far == max_ending_here:
                if i + 1 - ind0_starting_here > L:
                    ind0_max_so_far = ind0_starting_here
                    L = i + 1 - ind0_max_so_far

            i += 1
            if i < len(Arr):
                max_ending_here += Arr[i]
            else:
                max_ending_here = -1
        
    return max_so_far, L, ind0_max_so_far
    
@jit(nopython = True, parallel = True)
def Kadane_parallel(Arr, List, N_cores):
    N = len(Arr)//N_cores + 1
    Maxs, Ls, Inds = List.copy(), List.copy(), List.copy()
    for i in prange(N_cores):
        Maxs[i], Ls[i], Inds[i] = Kadene_job(Arr[i*N:], N)

    Max = 0
    L = 0
    ind0 = 0
    for i in range(N_cores):
        if Maxs[i] > Max:
            Max = Maxs[i]
            ind0 = Inds[i]
            L = Ls[i]
            
        elif Maxs[i] == Max and Ls[i] > L:
            ind0 = Inds[i]
            L = Ls[i]
            
    return Max, Arr[ind0:ind0+L]

In [48]:
# List of 4_000_000_000 numbers ranging from -20 to 19
numbers = np.random.randint(-20, 20, 4_000_000_000, dtype = np.int8)

start = time()
a, b = Kadane_numba(numbers)
print("Numba without parallellisation:", time() - start)

N_cores = 8
start = time()
c, d = Kadane_parallel(numbers, np.ones(N_cores, dtype = np.int64), N_cores)
print("Numba with parallellisation:", time() - start)

print(a, c)
print(len(b), len(d))

Numba without parallellisation: 7.450134992599487
Numba with parallellisation: 0.9979135990142822
2197 2197
2586 2586


In [45]:
147/19.7, 784/124, 953/158, 937/125, 565/353

(7.461928934010152,
 6.32258064516129,
 6.031645569620253,
 7.496,
 1.6005665722379603)

In [22]:
numbers = np.random.randint(-20, 21, 100)

print(Kadene_parallel(numbers, np.ones(8, dtype = np.int64), 8))
print(Kadene_extended(numbers))



(201, array([ 20,   0, -19,  15,  12,  15, -13,  11,  18, -15,  17,  15, -14,
        -3,   7,  17,  -6,  -1,  -8,  18,  15, -17,   4,  17,  -5,  -9,
        18,  15, -14,  16,  13,  -6,   1, -19,  15,   7,  19,  16,  12,
        -8,  -1,   7,   1, -20,  14, -20,   7,   7,   4,   6, -17,  13,
         9,  -3,  -9,   7,   8, -12,   8,  16]))
(201, array([ 20,   0, -19,  15,  12,  15, -13,  11,  18, -15,  17,  15, -14,
        -3,   7,  17,  -6,  -1,  -8,  18,  15, -17,   4,  17,  -5,  -9,
        18,  15, -14,  16,  13,  -6,   1, -19,  15,   7,  19,  16,  12,
        -8,  -1,   7,   1, -20,  14, -20,   7,   7,   4,   6, -17,  13,
         9,  -3,  -9,   7,   8, -12,   8,  16]))


In [13]:
a = np.random.randint(-20, 21, 10, dtype = np.int8)
a

array([  9,   9,   4,   4,  16,   7, -18, -17,  17, -16], dtype=int8)

In [32]:
def LSS_original(arr):
    new_arr = []
    for i in range(0, len(arr)):
        b =[]
        for j in range(i, len(arr)):
            b.append(np.sum(arr[j:j+1]))
            new_arr.append(b[:])
    sum_arr = []
    for h in new_arr:
        sum_arr.append(sum(h))
        indices = [index for index, value in enumerate(sum_arr) if value == max(sum_arr)]
    length = [len(new_arr[i]) for i in indices]
    p = length.index(max(length))
    winner = new_arr[indices[p]]
#     print('New Array =' , new_arr )
#     return print('Largest Sum =' , max(sum_arr), ', Longest Sequence =' , winner)
    return max(sum_arr), winner

In [38]:
def lss_final(arr):
    subseqs = [arr[i:j+1] for i in range(len(arr)) for j in range(i, len(arr))]
    subseq_sums = [sum(h) for h in subseqs]
    max_sum = max(subseq_sums)
    max_sum_subseqs = [subseq for subseq in subseqs if sum(subseq) == max_sum]
    max_length_of_max_sum_subseqs = max([len(subseq) for subseq in max_sum_subseqs])
    longest_max_sum_subseq = [subseq for subseq in max_sum_subseqs if len(subseq) == max_length_of_max_sum_subseqs][0]
#     return print('Array ', arr, ', Largest Sum =', max_sum, ', Longest Sequence =', longest_max_sum_subseq)
    return max_sum, longest_max_sum_subseq

In [53]:
#List of 70 numbers ranging from -20 to 20
numbers = np.random.randint(-20, 21, 70)

for function in LSS_original, lss_final, Kadene_extended, Kadene_numba:
    start_time = time()
    function(numbers)
    duration = time() - start_time
    print(f'{function.__name__} took {duration:0.5f} seconds')

LSS_original took 89.16619 seconds
lss_final took 0.02697 seconds
Kadene_extended took 0.00000 seconds
Kadene_numba took 0.00000 seconds


In [61]:
#List of 1000 numbers ranging from -20 to 20
numbers = np.random.randint(-20, 21, 1000)

for function in lss_final, Kadene_extended, Kadene_numba:
    start_time = time()
    function(numbers)
    duration = time() - start_time
    print(f'{function.__name__} took {duration:0.5f} seconds')

lss_final took 56.76192 seconds
Kadene_extended took 0.00100 seconds
Kadene_numba took 0.00000 seconds


In [69]:
#List of 100_000_000 numbers ranging from -20 to 20
numbers = np.random.randint(-20, 21, 100_000_000)

for function in Kadene_extended, Kadene_numba:
    start_time = time()
    function(numbers)
    duration = time() - start_time
    print(f'{function.__name__} took {duration:0.5f} seconds')

Kadene_extended took 60.80194 seconds
Kadene_numba took 0.14814 seconds


In [78]:
#List of 2_000_000_000 numbers ranging from -20 to 20
numbers = np.random.randint(-20, 21, 2_000_000_000)

start_time = time()
Kadene_numba(numbers)
duration = time() - start_time
print(f'{function.__name__} took {duration:0.5f} seconds')

Kadene_numba took 3.41879 seconds


In [83]:
numbers = np.random.randint(-20, 21, 2_000_000_000)

Sum, List = Kadene_numba(numbers)
print(Sum, len(List))

340973 486241573


In [11]:
A = [4,5,-12,1,10]
B = [1,2,-3,1,2]
# 
print(Kadene_extended(A))
print(Kadene_extended(B))

(11, [1, 10])
(3, [1, 2, -3, 1, 2])
